In [1]:
import sqlite3

# Conociendo la base de datos

In [2]:
def imprime_datos(BD_name): # Se debe ejecutar con la base de datos abierta.
    sql_statement = "SELECT name FROM sqlite_master WHERE type = 'table';"
    cursor.execute(sql_statement)
    tablas = [nombre[0] for nombre in cursor.fetchall()]

    print(f"Información base de datos: {BD_name}")
    for tabla in tablas:
        print(2*'\n'+ f"Información tabla {tabla}:")
        sql_statement = "PRAGMA table_info({})".format(tabla) #cid  name  type  notnull  dflt_value  pk
        cursor.execute(sql_statement)

        for columna in cursor:
            cid, name, tipo, notnull, dflt_val, pk = *columna,
            print(f'\tInformación columna {cid} - {name}: {tipo}, {notnull}, {dflt_val}, {pk}')

In [3]:
BD_name = 'lauretes.db'
connection = sqlite3.connect(BD_name)
cursor = connection.cursor()
imprime_datos(BD_name)
connection.commit()
connection.close()

Información base de datos: lauretes.db


Información tabla Persons:
	Información columna 0 - pid: INTEGER, 0, None, 1
	Información columna 1 - firstname: TEXT, 0, None, 0
	Información columna 2 - surname: TEXT, 0, None, 0
	Información columna 3 - born: DATE, 0, None, 0
	Información columna 4 - died: DATE, 0, None, 0
	Información columna 5 - bornCountry: TEXT, 0, None, 0
	Información columna 6 - bornCountryCode: TEXT, 0, None, 0
	Información columna 7 - bornCity: TEXT, 0, None, 0
	Información columna 8 - diedCountry: TEXT, 0, None, 0
	Información columna 9 - diedCountryCode: TEXT, 0, None, 0
	Información columna 10 - diedCity: TEXT, 0, None, 0
	Información columna 11 - gender: TEXT, 0, None, 0


Información tabla Prizes:
	Información columna 0 - prid: INTEGER, 0, None, 1
	Información columna 1 - year: INTEGER, 0, None, 0
	Información columna 2 - category: TEXT, 0, None, 0
	Información columna 3 - share: INTEGER, 0, None, 0
	Información columna 4 - motivation: TEXT, 0, None, 0


Informac

In [4]:
import json
import sqlite3

In [5]:
with open('laureates.json', encoding='utf8') as movies_file:
    lauretes = json.load(movies_file)

In [8]:
connection = sqlite3.connect('lauretes.db')
cursor = connection.cursor()

cursor.execute("CREATE TABLE IF NOT EXISTS Persons(pid INTEGER PRIMARY KEY, firstname TEXT, surname TEXT, born DATE, died DATE, bornCountry TEXT, bornCountryCode TEXT, bornCity TEXT, diedCountry TEXT, diedCountryCode TEXT, diedCity TEXT, gender TEXT)")
cursor.execute("CREATE TABLE IF NOT EXISTS Prizes(prid INTEGER PRIMARY KEY, year INTEGER, category TEXT, share INTEGER, motivation TEXT)")
cursor.execute("CREATE TABLE IF NOT EXISTS Affiliations(aid INTEGER PRIMARY KEY, name TEXT, city TEXT, contry TEXT)")
cursor.execute("CREATE TABLE IF NOT EXISTS PrizesAffilations(prize_id INTEGER, affiliation_id INTEGER, FOREIGN KEY (prize_id) REFERENCES Prizes, FOREIGN KEY (affiliation_id) REFERENCES Affiliations, PRIMARY KEY(prize_id, affiliation_id))")
cursor.execute("CREATE TABLE IF NOT EXISTS PersonsPrizes(person_id INTEGER, prize_id INTEGER, FOREIGN KEY (person_id) REFERENCES Persons, FOREIGN KEY (prize_id) REFERENCES Prizes, PRIMARY KEY(person_id, prize_id))")

connection.commit()
connection.close()

In [7]:
connection = sqlite3.connect('lauretes.db')
cursor = connection.cursor()

cursor.execute("DROP TABLE Persons")
cursor.execute("DROP TABLE Prizes")
cursor.execute("DROP TABLE PrizesAffilations")
cursor.execute("DROP TABLE Affiliations")
cursor.execute("DROP TABLE PersonsPrizes")


connection.commit()
connection.close()

In [9]:
connection = sqlite3.connect('lauretes.db')
cursor = connection.cursor()

prid = 1
aid = 1
prizes = {}
affiliations = {}

for laurete in lauretes:
    #Dejamos en nulo los datos faltantes, de ser necesario.
    if "diedCity" not in laurete.keys():
        laurete["diedCity"]= None
    if "diedCountry" not in laurete.keys():
        laurete["diedCountry"]= None
    if "diedCountryCode" not in laurete.keys():
        laurete["diedCountryCode"]= None
    if "surname" not in laurete.keys():
        laurete["surname"]= None
    if "bornCountry" not in laurete.keys():
        laurete["bornCountry"]= None
    if "bornCountryCode" not in laurete.keys():
        laurete["bornCountryCode"]= None
    if "bornCity" not in laurete.keys():
        laurete["bornCity"]= None
    if "born" not in laurete.keys():
        laurete['born']= None
    cursor.execute("INSERT INTO Persons VALUES (?,?,?,?,?,?,?,?,?,?,?,?)", (laurete['id'], 
                                                                            laurete['firstname'], 
                                                                            laurete['surname'], 
                                                                            laurete['born'], 
                                                                            laurete['died'], 
                                                                            laurete['bornCountry'], 
                                                                            laurete['bornCountryCode'], 
                                                                            laurete['bornCity'], 
                                                                            laurete['diedCountry'], 
                                                                            laurete['diedCountryCode'], 
                                                                            laurete['diedCity'], 
                                                                            laurete['gender']))
     
    for prize in laurete['prizes']:
        
        tuple_prize= (prize["year"], prize["category"], prize["motivation"])
        
        if tuple_prize not in prizes:
            
            cursor.execute("INSERT INTO Prizes VALUES (?,?,?,?,?)", (prid, 
                                                                     prize['year'], 
                                                                     prize['category'], 
                                                                     prize['share'], 
                                                                     prize['motivation']))
            prizes[tuple_prize]= prid
            #revisamos sus afiliaciones
            for affiliation in prize['affiliations']:
                if type(affiliation)!=list:
                    #null de datos faltantes
                    if "city" not in affiliation.keys():
                        affiliation['city']= None
                    if "country" not in affiliation.keys():
                        affiliation['country']= None  

                    affiliation_name= affiliation['name'] 
                    if affiliation_name not in affiliations:
                        affiliations[affiliation_name] = aid

                        cursor.execute("INSERT INTO Affiliations VALUES (?,?,?,?)", (aid, affiliation['name'], affiliation['city'], affiliation['country']))

                        aid = aid +  1
                    cursor.execute("INSERT INTO PrizesAffilations VALUES (?,?)", (prid, affiliations[affiliation_name]))
            
            prid= prid + 1
        cursor.execute("INSERT INTO PersonsPrizes VALUES (?,?)", (laurete["id"], prizes[tuple_prize]))
        
        
       
        

connection.commit()
connection.close()